# Problema

Imagine que você foi escalado para um time de investimentos e precisará realizar um modelo preditivo com dados da IBOVESPA (Bolsa de valores) para criar uma série temporal e prever diariamente o fechamento da
base.

Para isso utilize a base de dados contida no site da investing e selecione o período "diário", com o intervalo
de tempo que achar adequado.

Você precisará demonstrar para o time de investimentos:

*   O modelo com o storytelling, desde a captura do dado até a entrega do modelo;
*   Justificar a técnica utilizada;
*   Atingir uma acuracidade adequada (acima de 70%).